In [1]:
import numpy as np
import pandas as pd
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, GRU, BatchNormalization, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2
from Q3_DataLoader import RNN_load_data, trim_data_to_min_file_length
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Length of the input sequence during training
timesteps = 64
data_coeff = 1
num_bins = 3

# Data Sets
#REV 1
dataSet1 = "Norm400Hz_005_AA.csv"
dataSet2 = "Norm400Hz_006_BB.csv" #BAD DATASET - AltHold @ 4m
dataSet3 = "Norm400Hz_007_AA.csv"
dataSet4 = "Norm400Hz_008_AA.csv"
dataSet5 = "Norm400Hz_009_CC.csv" #SHORT
dataSet6 = "Norm400Hz_010_AA.csv"
dataSet7 = "Norm400Hz_014_AA.csv" #BAD DATASET - No 3D display
dataSet8 = "Norm400Hz_015_AA.csv"
dataSet9 = "Norm400Hz_016_BB.csv" #SHORT
dataSet10 = "Norm400Hz_019_AA.csv" #BAD DATASET
dataSet11 = "Norm400Hz_020_CC.csv" #BAD DATASET - Flying at .3m? 
dataSet12 = "Norm400Hz_021_CC.csv"
dataSet13 = "Norm400Hz_022_AA.csv"
dataSet14 = "Norm400Hz_023_AA.csv"
#REV 2
dataSet15 = "Norm400Hz_025_AA.csv" #BAD DATASET - No 3D display
dataSet16 = "Norm400Hz_026_CC.csv"
dataSet17 = "Norm400Hz_027_CC.csv" #DROPOUT
dataSet18 = "Norm400Hz_028_AA.csv" #SHORT
#REV 3
dataSet19 = "Norm400Hz_029_AA.csv" #BAD DATASET
dataSet20 = "Norm400Hz_030_AA.csv"
dataSet21 = "Norm400Hz_031_AA.csv"
dataSet22 = "Norm400Hz_032_AA.csv"
dataSet23 = "Norm400Hz_033_AA.csv"
dataSet24 = "Norm400Hz_034_AA.csv"
dataSet25 = "Norm400Hz_035_AA.csv"
dataSet26 = "Norm400Hz_049_AA.csv"
dataSet27 = "Norm400Hz_051_AA.csv"
dataSet28 = "Norm400Hz_053_AA.csv"
dataSet29 = "Norm400Hz_054_AA.csv"
dataSet30 = "Norm400Hz_055_AA.csv"
dataSet31 = "Norm400Hz_056_AA.csv"
dataSet32 = "Norm400Hz_057_AA.csv"
dataSet33 = "Norm400Hz_058_AA.csv"
dataSet34 = "Norm400Hz_059_AA.csv"
dataSet35 = "Norm400Hz_060_AA.csv"
dataSet36 = "Norm400Hz_061_AA.csv"
dataSet37 = "Norm400Hz_062_BB.csv"
dataSet38 = "Norm400Hz_065_BB.csv"
dataSet39 = "Norm400Hz_066_CC.csv"
dataSet40 = "Norm400Hz_067_BB.csv" #Already trimmed to avoid dropout 
dataSet41 = "Norm400Hz_068_BB.csv"
dataSet42 = "Norm400Hz_069_BB.csv"
dataSet43 = "Norm400Hz_070_CC.csv"
dataSet44 = "Norm400Hz_071_CC.csv"
dataSet45 = "Norm400Hz_072_BB.csv"
dataSet46 = "Norm400Hz_073_BB.csv"
dataSet47 = "Norm400Hz_074_CC.csv"
dataSet48 = "Norm400Hz_075_CC.csv"
dataSet49 = "Norm400Hz_076_CC.csv"
#DataSets with ATT, IMU_0, RCOU
dataSet50 = "Norm400Hz_Exp_030_AA.csv"
dataSet51 = "Norm400Hz_Exp_031_AA.csv"
dataSet52 = "Norm400Hz_Exp_032_AA.csv"
dataSet53 = "Norm400Hz_Exp_033_AA.csv"
dataSet54 = "Norm400Hz_Exp_035_AA.csv"
dataSet55 = "Norm400Hz_Exp_051_AA.csv"
dataSet56 = "Norm400Hz_Exp_054_AA.csv"
dataSet57 = "Norm400Hz_Exp_056_AA.csv"
dataSet58 = "Norm400Hz_Exp_062_BB.csv"
dataSet59 = "Norm400Hz_Exp_065_BB.csv"
dataSet60 = "Norm400Hz_Exp_066_CC.csv"
dataSet61 = "Norm400Hz_Exp_067_BB.csv"
dataSet62 = "Norm400Hz_Exp_068_BB.csv"
dataSet63 = "Norm400Hz_Exp_069_BB.csv"
dataSet64 = "Norm400Hz_Exp_070_CC.csv"
dataSet65 = "Norm400Hz_Exp_071_CC.csv"
dataSet66 = "Norm400Hz_Exp_072_BB.csv"
dataSet67 = "Norm400Hz_Exp_073_BB.csv"
dataSet68 = "Norm400Hz_Exp_074_CC.csv"
dataSet69 = "Norm400Hz_Exp_075_CC.csv"
dataSet70 = "Norm400Hz_Exp_076_CC.csv"

# # DataSets with only IMU_0, and RCOU
# # Categorize the datasets. This list is used for training
# datasets_AA = [dataSet20, dataSet21, dataSet22, dataSet23, dataSet24]#, dataSet25, dataSet26, dataSet27, dataSet28, dataSet29, dataSet30, dataSet31, dataSet32]#, dataSet33, dataSet34, dataSet35, dataSet36]
# datasets_BB = [dataSet37, dataSet38, dataSet40, dataSet41, dataSet42]#, dataSet45]#, dataSet46]
# datasets_CC = [dataSet39, dataSet43, dataSet44, dataSet47, dataSet48]#, dataSet49]

# # Do not train on this data. Use it for predictions
# predict_AA = [dataSet33, dataSet34, dataSet35, dataSet36]
# predict_BB = [dataSet46]#, dataSet22, dataSet30]
# predict_CC = [dataSet49]#, dataSet16, dataSet32]

# DataSets with ATT, IMU_0, and RCOU
datasets_AA = [dataSet50, dataSet51, dataSet52, dataSet53, dataSet54]#, dataSet55, dataSet56, dataSet57]
datasets_BB = [dataSet58, dataSet59, dataSet61, dataSet62, dataSet63]#, dataSet66, dataSet67]
datasets_CC = [dataSet60, dataSet64, dataSet65, dataSet68, dataSet69]#, dataSet70]
datasets_DD = []
datasets_EE = []

# Shuffle the datasets in each category so that the model does not always train off of the earliest flights
# random.shuffle(datasets_AA)
# random.shuffle(datasets_BB)
# random.shuffle(datasets_CC)

# Find the smallest category
min_category_size = min(len(datasets_AA), len(datasets_BB), len(datasets_CC))#, len(datasets_DD), len(datasets_EE))

# Trim each category to the size of the smallest category
datasets_AA = datasets_AA[:min_category_size]
datasets_BB = datasets_BB[:min_category_size]
datasets_CC = datasets_CC[:min_category_size]
datasets_DD = datasets_DD[:min_category_size]
datasets_EE = datasets_EE[:min_category_size]


# Combine the data file names into categories
data_categories = [datasets_AA, datasets_BB, datasets_CC]#, datasets_DD, datasets_EE]

# Trim the data in each category to the minimum total length
dataSets = trim_data_to_min_file_length(data_categories)

# Define the neural network model / Last layer needs to match number of bins
model_cg = Sequential([
    GRU(units=16, activation='tanh', return_sequences=True, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
    Dropout(0.05),
    GRU(units=8, activation='tanh', return_sequences=False, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
    Dropout(0.05),
    Dense(units=num_bins, activation='softmax')  
    # GRU(units=128, activation='tanh', return_sequences=True, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
    # Dropout(0.2),
    # GRU(units=64, activation='tanh', return_sequences=False, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
    # Dropout(0.2),
    # Dense(units=32, activation='relu'),
    # Dense(units=3, activation='softmax') 
])

# Compile the model
opt = Adam(learning_rate=0.001)
model_cg.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='accuracy', patience=3)
lr_reduction = ReduceLROnPlateau(monitor='loss', patience=2, verbose=1, factor=0.5)
model_checkpoint_acc = ModelCheckpoint('CG_Model.h5', save_best_only=True, monitor='accuracy', mode='max', verbose=1)

print(dataSets[0][0], "Loaded")
inputs, outputs = RNN_load_data(dataSets[0][0], timesteps, data_coeff)

first_dataset_loaded = False
for category in dataSets:
    for dataSet in category:
        if not first_dataset_loaded:
            first_dataset_loaded = True
            continue
        print(dataSet, "Loaded")
        inputData, outputData = RNN_load_data(dataSet, timesteps, data_coeff)
        inputs = np.concatenate((inputs, inputData), axis=0)
        outputs = np.concatenate((outputs, outputData), axis=0)
print("Data Loading Finished")
print("Data Length:", len(inputs))

# Train the model
model_cg.fit(inputs, outputs, epochs=25, batch_size=1024, callbacks=[early_stopping, lr_reduction, model_checkpoint_acc])

# Save the model (Unnecessary because the ModelCheckpoint callback already saves it)
# model_cg.save('CG_Model.h5')
print("Model Saved")

TempTrimmed_Norm400Hz_Exp_030_AA.csv Loaded
Data Set Retrieved
TempTrimmed_Norm400Hz_Exp_031_AA.csv Loaded
Data Set Retrieved
TempTrimmed_Norm400Hz_Exp_032_AA.csv Loaded
Data Set Retrieved
TempTrimmed_Norm400Hz_Exp_033_AA.csv Loaded
Data Set Retrieved
TempTrimmed_Norm400Hz_Exp_035_AA.csv Loaded
Data Set Retrieved
TempTrimmed_Norm400Hz_Exp_062_BB.csv Loaded
Data Set Retrieved
TempTrimmed_Norm400Hz_Exp_065_BB.csv Loaded
Data Set Retrieved
TempTrimmed_Norm400Hz_Exp_067_BB.csv Loaded
Data Set Retrieved
TempTrimmed_Norm400Hz_Exp_068_BB.csv Loaded
Data Set Retrieved
TempTrimmed_Norm400Hz_Exp_069_BB.csv Loaded
Data Set Retrieved
TempTrimmed_Norm400Hz_Exp_066_CC.csv Loaded
Data Set Retrieved
TempTrimmed_Norm400Hz_Exp_070_CC.csv Loaded
Data Set Retrieved
TempTrimmed_Norm400Hz_Exp_071_CC.csv Loaded
Data Set Retrieved
TempTrimmed_Norm400Hz_Exp_074_CC.csv Loaded
Data Set Retrieved
TempTrimmed_Norm400Hz_Exp_075_CC.csv Loaded
Data Set Retrieved
Data Loading Finished
Data Length: 1703460
Epoch 1/25


I0000 00:00:1712084376.011627   65903 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1662/1664 [============================>.] - ETA: 0s - loss: 0.9218 - accuracy: 0.5755
Epoch 1: accuracy improved from -inf to 0.57550, saving model to CG_Model.h5
1664/1664 [==============================] - 20s 10ms/step - loss: 0.9217 - accuracy: 0.5755 - lr: 0.0010
Epoch 2/25
  16/1664 [..............................] - ETA: 16s - loss: 0.7854 - accuracy: 0.6304

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1659/1664 [============================>.] - ETA: 0s - loss: 0.7140 - accuracy: 0.6567
Epoch 2: accuracy improved from 0.57550 to 0.65673, saving model to CG_Model.h5
1664/1664 [==============================] - 17s 10ms/step - loss: 0.7139 - accuracy: 0.6567 - lr: 0.0010
Epoch 3/25
1659/1664 [============================>.] - ETA: 0s - loss: 0.6397 - accuracy: 0.7000
Epoch 3: accuracy improved from 0.65673 to 0.70002, saving model to CG_Model.h5
1664/1664 [==============================] - 16s 10ms/step - loss: 0.6396 - accuracy: 0.7000 - lr: 0.0010
Epoch 4/25
1662/1664 [============================>.] - ETA: 0s - loss: 0.5995 - accuracy: 0.7242
Epoch 4: accuracy improved from 0.70002 to 0.72421, saving model to CG_Model.h5
1664/1664 [==============================] - 16s 10ms/step - loss: 0.5995 - accuracy: 0.7242 - lr: 0.0010
Epoch 5/25
1663/1664 [============================>.] - ETA: 0s - loss: 0.5707 - accuracy: 0.7350
Epoch 5: accuracy improved from 0.72421 to 0.73505, saving mo

In [4]:
import os
from tensorflow.keras.models import load_model
os.chdir('/home/coder/workspace/Data/Becky_Data/')

# Load the previously trained model
model_cg = load_model('CG_Model.h5')

# Continue training the model
model_cg.fit(inputs, outputs, epochs=10, batch_size=2024, callbacks=[early_stopping, lr_reduction, model_checkpoint_acc])

print("Model Saved")

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.